In [ ]:
%pwd
%cd /app

import os

import dac
from audiotools import AudioSignal

import torch

import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as ipd

import math
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.device_count()
torch.cuda.get_device_properties(0).total_memory/1e9
device = torch.device('cuda') # if the docker was started with --gpus all, then can choose here with cuda:0 (or cpu)
device

torch.cuda.device_count()
print(f'memeory on cuda 0 is  {torch.cuda.get_device_properties(0).total_memory/1e9}')

In [ ]:
model_path = dac.utils.download(model_type="44khz") 
model = dac.DAC.load(model_path)

model.to(device); #wanna see the model? remove the semicolon
model.eval();  # need to be "in eval mode" in order to set the number of quantizers

In [ ]:
import os

def compress_wav_files(input_dir, output_dir, N_QUANTIZERS=9):
    """
    Processes all .wav files from the input directory and writes the results with a .dac extension to the output directory.

    Args:
    input_dir (str): Directory containing .wav files.
    output_dir (str): Directory where .dac files will be saved.
    """
    # Check if output directory exists, create if not
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate through all files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".wav"):
            # Full path for input and output files
            input_file = os.path.join(input_dir, filename)

            signal = AudioSignal(input_file)
            x = model.compress(signal, n_quantizers=N_QUANTIZERS)
            

            # Change the file extension from .wav to .dac
            output_file = os.path.join(output_dir, os.path.splitext(filename)[0] + ".dac")

            # Save and load to and from disk
            x.save(output_file)
            
            print(f"Processed and saved {output_file}")

# Example usage
compress_wav_files('/scratch/dacdevdata/44kHz/N4/PisWinAppBee_sparse_recon', '/scratch/foo')

In [ ]:
#test
# load the file, and then 
with torch.no_grad():
    dacfile = dac.DACFile.load("/scratch/foo/DSApplause--numClappers_exp-00.50.dac")
    
    # FIRST - Decompress it back to an AudioSignal\ from codes to z (1024) to signal   
    print(f'dacfile.codes shape is: {dacfile.codes.shape}')
    z_from_c, l_from_c,c =model.quantizer.from_codes(dacfile.codes.to(device))
    print(f'z_from_c shape is: {z_from_c.shape}')
    print(f'l_from_c shape is: {l_from_c.shape}')
    print(f'c shape is: {c.shape}')
    xTensor = model.decode(z_from_c)

    # SECOND - Decompress directly using model.decompress   
    yAudioSig = model.decompress(dacfile)  #an AudioSignal

    
print(f' The length of  xTensor  signal is {len(xTensor.cpu().detach().numpy()[0,0,:])}')
print(f' The length of yAudioSig signal is {len(yAudioSig.cpu().detach().numpy()[0,0,:])}')

In [ ]:
xsig=xTensor.cpu().detach().numpy()[0,0,:]
plt.plot(xsig)
ipd.Audio(xsig, rate=44100)

In [ ]:


ysig=yAudioSig.cpu().detach().numpy()[0,0,:]
plt.plot(ysig)
ipd.Audio(ysig, rate=44100)


In [ ]:
len(xsig)

In [ ]:
len(ysig)